<a href="https://colab.research.google.com/github/rabimba/GDE-ML-Artifacts/blob/main/Gemini_Function_Calling_for_Structured_Data_Responses.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gemini Function Calling for Structured Data Responses

## Authentication

In [ ]:
# Authenticate via Colab
import sys
if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

# Define project/location and initialize Vertex AI

In [ ]:
# Define project information
PROJECT_ID = "gde-all-rk"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

# Initialize Vertex AI
import vertexai
vertexai.init(project=PROJECT_ID, location=LOCATION)

## Imports

In [ ]:
import json
from vertexai.preview.generative_models import (
    FunctionDeclaration,
    GenerativeModel,
    Tool,
)

model = GenerativeModel("gemini-1.0-pro")

## Method 1: Prompting for structured JSON output (without Function Calling)

In [ ]:
animal_name = "hummingbird"

prompt = f"""
You are a professional photographer, and will publish a photo book of
{animal_name}. What kind of photos and scenes will you choose to make the photo
book attractive and fun that focuses on their facial features, habitats,
families and herds, in different scenes and seasons?

Output the 10 words of titles and 20 words of detailed description of the
photos, in a JSON format like this:

[
  {{"title": "close-up portraits of a {animal_name} face", "description": "the intricate
details of {animal_name}'s facial features, including its eyes, nose, and mouth"}},
  {{"title": "wide-angle shots of an {animal_name} and their herds in its natural habitat",
  "description": "the vastness of the {animal_name}'s territory, showcasing its freedom and
  agility"}}
]
"""

response = model.generate_content(
  prompt,
  generation_config={
      "max_output_tokens": 2048,
      "temperature": 0.9,
      "top_p": 1
  },
)

output = json.loads(response.text)
output

[{'title': 'close-up portraits of a hummingbird face',
  'description': "the intricate details of hummingbird's facial features, including its eyes, nose, and mouth"},
 {'title': 'wide-angle shots of an hummingbird and their herds in its natural habitat',
  'description': "the vastness of the hummingbird's territory, showcasing its freedom and agility"},
 {'title': 'a family of hummingbirds',
  'description': 'the relationships within a hummingbird family, capturing their interactions and bonds'},
 {'title': 'the courtship dance of a pair of hummingbirds',
  'description': 'the graceful and delicate movements of hummingbirds during their courtship rituals'},
 {'title': 'the nest of a hummingbird',
  'description': "the intricate structure of a hummingbird's nest, showcasing their engineering skills"},
 {'title': 'a hummingbird in flight',
  'description': 'the dynamic and graceful movements of a hummingbird in mid-air'},
 {'title': 'a hummingbird feeding',
  'description': 'the delicat

## Method 2: Using Function Calling to output structured data

In [ ]:
# @title Helper functions to parse function call response
from vertexai.preview.generative_models import GenerativeModel, GenerationResponse
from proto.marshal.collections import repeated
from proto.marshal.collections import maps

def recurse_proto_repeated_composite(repeated_object):
    repeated_list = []
    for item in repeated_object:
        if isinstance(item, repeated.RepeatedComposite):
            item = recurse_proto_repeated_composite(item)
            repeated_list.append(item)
        elif isinstance(item, maps.MapComposite):
            item = recurse_proto_marshal_to_dict(item)
            repeated_list.append(item)
        else:
            repeated_list.append(item)

    return repeated_list

def recurse_proto_marshal_to_dict(marshal_object):
    new_dict = {}
    for k, v in marshal_object.items():
      if not v:
        continue
      elif isinstance(v, maps.MapComposite):
          v = recurse_proto_marshal_to_dict(v)
      elif isinstance(v, repeated.RepeatedComposite):
          v = recurse_proto_repeated_composite(v)
      new_dict[k] = v

    return new_dict

def get_text(response: GenerationResponse):
  """Returns the Text from the Generation Response object."""
  part = response.candidates[0].content.parts[0]
  try:
    text = part.text
  except:
    text = None

  return text

def get_function_name(response: GenerationResponse):
  return response.candidates[0].content.parts[0].function_call.name

def get_function_args(response: GenerationResponse) -> dict:
  return recurse_proto_marshal_to_dict(response.candidates[0].content.parts[0].function_call.args)

Define a Function Decalaration:

In [ ]:
generate_photograph_content_func = FunctionDeclaration(
    name="generate_photograph_content",
    description="""
    You are a professional photographer, and will publish a photo book of
    an {animal_name}. What kind of photos and scenes will you choose to make the
    photo book attractive and fun that focuses on their facial features,
    habitats, families and herds, in different scenes and seasons?

    Generate about 10 words for the title and 20 words for a detailed
    description of the photos""",
    parameters={
    "type": "object",
    "properties": {
      "photos": {
        "type": "array",
        "description": "A list of photo titles and descriptions",
        "items": {
          "description": "Photo details",
          "type": "object",
          "properties": {
            "title": {
                "type": "string",
                "description": "Title of the photo, such as: Close-up portraits of a {animal_name} face. Or wide-angle shots of an {animal_name} and their herds in its natural habitat."
              },
            "description": {
              "type": "string",
              "description": "Description of the photo, such as: the intricate details of {animal_name}'s facial features, including its eyes, nose, and mouth. Or the vastness of the {animal_name}'s territory, showcasing its freedom and agility"
            }
          }
        }
      }
    },
         "required": [
            "title",
            "description",
      ]
  },
)

Register the function as a `Tool`:

In [ ]:
generate_photograph_content_tool = Tool(
    function_declarations=[generate_photograph_content_func],
)

Send a prompt with the animal name and desired number of photos:

In [ ]:
prompt = """Generate about 10 options for a photo book about hummingbirds"""

response = model.generate_content(
    prompt,
    tools=[generate_photograph_content_tool],
)

In [ ]:
output = get_function_args(response)
output

{'photos': [{'title': 'Hummingbird in Flight',
   'description': 'Close-up of a hummingbird in mid-flight, with its wings extended and its beak pointed towards the ground'}]}

In [ ]:
for photo in output['photos']:
    print(photo["title"])
    print(photo["description"])
    print()

Hummingbird in Flight
Close-up of a hummingbird in mid-flight, with its wings extended and its beak pointed towards the ground

